In [35]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [20]:
df = pd.read_csv("../data/02/df_zillow.csv")
df.transactiondate = df.transactiondate.apply(lambda x: str(x)[5:7] + "-" + str(x)[0:4])

In [23]:
df_10_2016 = df.loc[df.transactiondate == "10-2016"]
df_11_2016 = df.loc[df.transactiondate == "11-2016"]
df_12_2016 = df.loc[df.transactiondate == "12-2016"]
df_10_2017 = df.loc[df.transactiondate == "10-2017"]
df_11_2017 = df.loc[df.transactiondate == "11-2017"]
df_12_2017 = df.loc[df.transactiondate == "12-2017"]

In [50]:
def myRF(df) :
    model = RandomForestRegressor(n_estimators=100, random_state=0)

    numerical_cols = []
    categorical_cols = []
    for x in df.columns :
        if x == "transactiondate" :
            pass
        elif df[x].dtype != "object" :
            numerical_cols.append(x)
        else :
            categorical_cols.append(x)

    
    # Preprocessing for numerical data
    numerical_transformer = SimpleImputer(strategy='constant')

    # Preprocessing for categorical data
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Bundle preprocessing for numerical and categorical data
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])

    # Bundle preprocessing and modeling code in a pipeline
    my_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    y = df.logerror
    X = df[numerical_cols]

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

    # Preprocessing of training data, fit model 
    my_pipeline.fit(X_train, y_train)

    # Preprocessing of validation data, get predictions
    preds = my_pipeline.predict(X_test)

    # Evaluate the model
    score = mean_absolute_error(y_test, preds)
    print('MAE:', score)

    return rf_model

In [51]:
rf_model_10_2016 = myRF(df_10_2016)

ValueError: A given column is not a column of the dataframe